# RHF 一阶梯度与二阶梯度

这一节我们开始了解 RHF 的一阶梯度与二阶梯度．这一部分的体量会很大；我们需要了解梯度的具体实现过程、数值梯度的实现、以及介绍在之后的工作中起到重要辅助作用的 HF 帮手类 HFHelper，包括其绝大多数变量的高级实现与底层实现．

这一节我们只讨论一些简单的 Skeleton 导数的问题．涉及 U 矩阵导数的问题，以及能量一阶、二阶梯度推导的原理，不在这一节的范畴内．

In [ ]:
import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft
from functools import partial

import sys
sys.path.append('../../src')
from utilities import val_from_fchk
from hf_helper import HFHelper
from numeric_helper import NumericDiff

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=150, suppress=True)

<div class="alert alert-warning">

**注意**

为了以后的阅读便利，我们可能会隐藏一些代码．下述代码块不论是 HF 方法还是 GGA 方法，都将会无条件地隐藏．

如果需要阅读被隐藏的代码，你可能需要阅读 Jupyter Notebook 本身；网页不会提供隐藏代码块的信息．

</div>

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! NOTE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For clear and simplicity, following code cell will be hidden and assumed to be executed!

mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

grids = dft.gen_grid.Grids(mol)
grids.atom_grid = (99, 590)
grids.becke_scheme = dft.gen_grid.stratmann
grids.prune = None
grids.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

## PySCF 一阶梯度计算

在生成一阶梯度前，我们需要使用 PySCF 跑一次自洽场过程．

In [ ]:
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-12
scf_eng.conv_tol_grad = 1e-10
scf_eng.kernel()

使用 PySCF 计算一阶梯度的代码是

In [ ]:
scf_grad = grad.RHF(scf_eng)
scf_grad.kernel()

上一个代码块中，输出即是一阶梯度了．计算结果储存在 `pyscf.grad.rhf.Gradients.de` 成员中．我们可以将其与 [Gaussian 结果](include/grad_rhf/rhf_grad.gjf) 进行比对：

In [ ]:
np.allclose(
    scf_grad.de.ravel(),
    val_from_fchk("Cartesian Gradient", "include/grad_rhf/rhf_grad.fchk")
)

另一种验证梯度结果是否正确的方法可以是进行数值梯度的计算．我们可以使用自定义的数值梯度帮手 `NumericDiff` 类来进行简单的数值计算．我们通过向该类提供能量计算函数，以及作为基础的分子类即可以得到数值能量．但对于当前体系，如果使用三点微分方法，则需要计算 24 个能量值；这需要耗费一些时间．

In [ ]:
def mol_to_eng(mol):
    mf = scf.RHF(mol)
    mf.conv_tol = 1e-12
    mf.conv_tol_grad = 1e-10
    mf.kernel()
    return mf.e_tot

In [ ]:
%%time
numeric_grad = NumericDiff(mol, mol_to_eng)
numeric_grad.get_numdif();

In [ ]:
np.allclose(
    numeric_grad.numdif,
    scf_grad.de
)

<div class="alert alert-info">

**任务**

1. (可选) 请自行查看代码库中的 `numeric_helper.py` 文件，了解如何进行数值梯度的计算；并且通过阅读程序文档 (Docstring)，尝试三点、五点差分选项，求取 Fock 矩阵的一阶数值梯度，以及该分子的 Hessian 矩阵．我们以后会经常使用这类数值梯度程序进行梯度计算正确性的验证．

2. 请判断梯度量 `scf_grad.de` 中，每个元素的量纲与单位．

   提示：Gaussian 的输出文件实际上已经写出了梯度的量纲与单位；但你可以通过思考数值梯度计算的过程，辅助验证是否是 Gaussian 的输出．

</div>

## PySCF 二阶梯度计算

PySCF 二阶梯度的代码实现是

In [ ]:
scf_hess = hessian.RHF(scf_eng)
scf_hess.kernel();

由于 Hessian 是一个原子数平方乘上 3 维度平方的张量，目前四原子的体系就有 144 个元素，已经比较大了，因此我们在这里静默 Hessian 的输出．

Hessian 值在 PySCF 中，储存在 `pyscf.hessian.rhf.Hessian.de` 中．我们也可以通过 [Gaussian 结果](include/grad_rhf/rhf_grad.gjf) 来比对 PySCF 所生成的 Hessian：

In [ ]:
d_hess = mol.natm * 3
print(np.allclose(
    scf_hess.de.swapaxes(1, 2).reshape(d_hess, d_hess)[np.tril_indices(d_hess)],
    val_from_fchk("Cartesian Force Constants", "include/grad_rhf/rhf_grad.fchk"),
    atol=1e-7
))

同样地，我们还可以使用对一阶梯度求数值梯度的方法来计算 Hessian：

In [ ]:
def mol_to_grad(mol):
    mf = scf.RHF(mol)
    mf.conv_tol = 1e-12
    mf.conv_tol_grad = 1e-10
    mf.kernel()
    mf_grad = grad.RHF(mf)
    return mf_grad.kernel()

In [ ]:
%%time
from numeric_helper import NumericDiff
numeric_hess = NumericDiff(mol, mol_to_grad, deriv=2)
numeric_hess.get_numdif();

In [ ]:
np.allclose(
    numeric_hess.numdif,
    scf_hess.de
)

<div class="alert alert-info">

**任务**

1. 请判断 Hessian 量中，每个元素的量纲与单位．

2. 在 Gaussian 中，Hessian 也称为 Force constants．请判断 Gaussian 所输出的力常数的每个元素的物理意义 (属于哪对原子的坐标分量的力常数贡献)．

   你也可以判断下述语句执行的意义，以辅助你的想法．
   
   ```
d_hess = mol.natm * 3
scf_hess.de.swapaxes(1, 2).reshape(d_hess, d_hess)[np.tril_indices(d_hess)]
   ```

</div>

## HF 帮手程序

HF 帮手程序是在笔记外面 `src` 文件夹下的 `hf_helper.py` 程序的类 `HFHelper` 所给出．在这里，我们先把这个程序所得到的输出信息列举一遍；这些输出信息还包括 HF 二阶梯度或二阶 U 矩阵相关的信息，这些信息会在以后的笔记中使用到．

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! NOTE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For clear and simplicity, following code cell will be hidden and assumed to be executed!

hfh = HFHelper(mol)

hfh.get_grad()
hfh.get_hess()

scf_eng  = hfh.scf_eng
scf_grad = hfh.scf_grad
scf_hess = hfh.scf_hess

C       = hfh.C
Co      = hfh.Co     
Cv      = hfh.Cv     
e       = hfh.e      
eo      = hfh.eo     
ev      = hfh.ev     
D       = hfh.D      
F_0_ao  = hfh.F_0_ao 
F_0_mo  = hfh.F_0_mo 
H_0_ao  = hfh.H_0_ao 
H_0_mo  = hfh.H_0_mo 
eri0_ao = hfh.eri0_ao
eri0_mo = hfh.eri0_mo
mo_occ  = hfh.scf_eng.mo_occ
H_1_ao  = hfh.get_H_1_ao  ()
H_1_mo  = hfh.get_H_1_mo  ()
S_1_ao  = hfh.get_S_1_ao  ()
S_1_mo  = hfh.get_S_1_mo  ()
F_1_ao  = hfh.get_F_1_ao  ()
F_1_mo  = hfh.get_F_1_mo  ()
eri1_ao = hfh.get_eri1_ao ()
eri1_mo = hfh.get_eri1_mo ()
H_2_ao  = hfh.get_H_2_ao  ()
H_2_mo  = hfh.get_H_2_mo  ()
S_2_ao  = hfh.get_S_2_ao  ()
S_2_mo  = hfh.get_S_2_mo  ()
F_2_ao  = hfh.get_F_2_ao  ()
F_2_mo  = hfh.get_F_2_mo  ()
eri2_ao = hfh.get_eri2_ao ()
eri2_mo = hfh.get_eri2_mo ()
B_1     = hfh.get_B_1     ()
U_1     = hfh.get_U_1     ()
Xi_2    = hfh.get_Xi_2    ()
B_2_vo  = hfh.get_B_2_vo  ()
U_2_vo  = hfh.get_U_2_vo  ()

Ax0_Core = hfh.Ax0_Core
Ax1_Core = hfh.Ax1_Core

上述程序中的 `C`, `Co`, `Cv`, `e`, `eo`, `ev`, `D`, `F_0_ao`, `F_0_mo`, `H_0_ao`, `H_0_mo`, `eri0_ao`, `eri0_mo`, `mo_occ` 等变量的含义已经在 [MP2 笔记](basic_mp2.ipynb#自洽场计算相关量) 中叙述过，这里就不再额外地说明．剩下的变量通常与梯度性质有关．对于梯度的公式，我们尽量让记号贴近 Yamaguchi et al <cite data-cite="Yamaguchi-Schaefer.Oxford.1994"></cite> 的记号；但多少会有些变动．

<div class="alert alert-info">

**记号说明**

* $A_t$ 代表某原子 $A$ 对应的坐标分量 $t$．在二阶梯度中，还会使用到 $B_s$．这将代替 Yamaguchi 书中被求导的 $a$ 与 $b$．

</div>

* `scf_eng` 为 RHF 能量计算类，这与以前的定义是相同的

* `scf_grad` 为 RHF 梯度计算类，与这一份笔记中出现过的 `scf_grad` 也是相同的

* `scf_hess` 为 RHF 二阶梯度计算类

* `H_1_ao` $h_{\mu \nu}^{A_t}$ 为 $h_{\mu \nu}$ 在原子坐标分量 $A_t$ 下的 Skeleton 导数；后同

* `H_1_mo` $h_{pq}^{A_t} = C_{\mu p} h_{\mu \nu}^{A_t} C_{\nu q}$，对于 Fock、重叠矩阵、双电子积分类同

* `S_1_ao` $S_{\mu \nu}^{A_t}$

* `S_1_mo` $S_{pq}^{A_t}$

* `F_1_ao` $F_{\mu \nu}^{A_t}$

* `F_1_mo` $F_{pq}^{A_t}$

* `eri1_ao` $(\mu \nu | \kappa \lambda)^{A_t}$

* `eri1_mo` $(p q | r s)^{A_t}$

* `H_2_ao` $h_{\mu \nu}^{A_t B_s}$

* `H_2_mo` $h_{pq}^{A_t B_s}$

* `S_2_ao` $S_{\mu \nu}^{A_t B_s}$

* `S_2_mo` $S_{pq}^{A_t B_s}$

* `F_2_ao` $F_{\mu \nu}^{A_t B_s}$

* `F_2_mo` $F_{pq}^{A_t B_s}$

* `eri2_ao` $(\mu \nu | \kappa \lambda)^{A_t B_s}$

* `eri2_mo` $(p q | r s)^{A_t B_s}$

* `B_1` $B_{pq}^{A_t}$，参见 Yamaguchi (p437, X.3) 的定义，但不使用 $B_{0, pq}^{A_t}$ 的记号；对于 $B_{pq}^{A_t B_s}$ 同理

* `U_1` $U_{pq}^{A_t}$

* `X1_2` $\xi_{pq}^{A_t B_s}$，参见 Yamaguchi (p405, L.4) 的定义

* `B_2_vo` $B_{ai}^{A_t B_s}$；这里并没有生成全轨道的张量，只有非占-占据的部分

* `U_2_vo` $U_{ai}^{A_t B_s}$；同样只生成了非占-占据部分

* `Ax0_Core` 函数用于计算 $A_{pq, rs} X_{rs}^{\cdots}$，该函数的调用将会在以后作说明

* `Ax1_Core` 函数用于计算 $A_{pq, rs}^{A_t} X_{rs}^{\cdots}$，该函数的调用将会在以后作说明

事实上，根据上面定义的矩阵与张量，我们已经能很容易地通过 Yamaguchi (p428, V.1, V.2) 来获得梯度与 Hessian 的结果了．

<div class="alert alert-info">

**任务**

1. 请对上面的一些变量查看其维度．

</div>

<div class="alert alert-warning">

**注意**

以后的笔记中，上述代码块中与自洽场有关的量，即系数矩阵 `C`, `Co`, `Cv`，轨道能 `e`, `eo`, `ev`，密度量 `D`，以及 U 矩阵 `U_1`, `U_2_vo` 都会隐藏起来；它们非常常用且与自洽、非自洽泛函是无关．其它变量名在只有自洽泛函的文档中会不加说明地使用；而若涉及到非自洽泛函，直接使用这些变量名的情况会在文档中作额外说明．

</div>

## 实现参考：能量梯度

### 电子态能量的一阶梯度 $\frac{\partial}{\partial A_t} E_\mathrm{elec}$

In [ ]:
scf_grad.grad_elec.__func__

Yamaguchi (p428, V.1)

$$
\frac{\partial}{\partial A_t} E_\mathrm{elec} = 2 h_{ii}^{A_t} + 2 (ii|jj)^{A_t} - (ij|ij)^{A_t} - 2 S_{ii}^{A_t} \varepsilon_i
$$

In [ ]:
np.allclose(
    + 2 * H_1_mo[:, :, so, so].trace(0, 2, 3)
    + 2 * eri1_mo[:, :, so, so, so, so].trace(0, 2, 3).trace(0, 2, 3)
    - eri1_mo[:, :, so, so, so, so].trace(0, 2, 4).trace(0, 2, 3)
    - 2 * (S_1_mo[:, :, so, so].diagonal(0, 2, 3) * eo).sum(axis=2),
    scf_grad.grad_elec()
)

事实上，有时用 AO 轨道来表述问题反而会在写程序时表现得更清楚，因为求迹的函数并不是那么直观；并且事实上使用 AO 轨道通常可以节省许多计算量．当然，从代码长度上，两者基本是一样的．

上述问题在 AO 轨道下的表述为 (涉及轨道能的部分仍然用分子轨道；但你也可以遵循大部分量化程序的做法，让 $S_{\mu \nu}^{A_t}$ 与轨道能加权密度矩阵 $D_{\mu \nu}[\varepsilon_i]$ 相乘；下述代码从计算量的角度来讲，与使用轨道能加权密度矩阵的大小是等价的)：

$$
\frac{\partial}{\partial A_t} E_\mathrm{elec} = h_{\mu \nu}^{A_t} D_{\mu \nu} + \frac{1}{2} (\mu \nu | \kappa \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda} - \frac{1}{4} (\mu \kappa | \nu \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda} - 2 S_{\mu \nu} C_{\mu i} \varepsilon_i C_{\nu i}
$$

In [ ]:
np.allclose(
    + np.einsum("Atuv, uv -> At", H_1_ao, D)
    + 0.5 * np.einsum("Atuvkl, uv, kl -> At", eri1_ao, D, D)
    - 0.25 * np.einsum("Atukvl, uv, kl -> At", eri1_ao, D, D)
    - 2 * np.einsum("Atuv, ui, i, vi -> At", S_1_ao, Co, eo, Co),
    scf_grad.grad_elec()
)

<div class="alert alert-info">

**任务**

1. 你可能会对刚才提到的 AO 表示比 MO 表示的计算量通常更省的说法有所疑问．

    1. 如果你没有产生疑问，请尝试对上述两个代码块进行 `%%timeit` 的运行时间估计．我们在 [numpy 介绍](python_intro.ipynb#矩阵乘积) 中已经使用过 `%%timeit` 指令；请学习 [IPython 网站](https://ipython.org/ipython-doc/3/interactive/magics.html#magic-timeit) 网站的帮助文档，调整 `-r`, `-n` 的数值为合理的数值．**不能照搬** [numpy 介绍](python_intro.ipynb#矩阵乘积) 中的代码，否则执行代码块的时间会很长．你应当会发现使用 AO 表示的计算耗时比 MO 表示其实还要大出 1.5 倍左右．表面上，AO 表示下的计算耗时更多．
   
    2. 我们分析为何我们还是要说 AO 表示下的计算量更低．上述计算中，耗时最大的显然是与双电子积分有关的计算．我们现在假设 $(\mu \nu | \kappa \lambda)^{A_t}$ 是必须要生成的；请分别指出对 $(\mu \nu | \kappa \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda}$ 求和，与生成 $(ii|jj)^{A_t}$ 这两个任务，最高计算复杂度的最低量级．
      
        注：我们把原子数、坐标分量也分别看成一个有效计算量级．当然，如果我们不考虑原子数与其坐标分量，这个问题其实也等价于普通的原子轨道与分子轨道的转换计算量问题．在学习 MP2 时，我们已经知道 $(\mu \nu | \kappa \lambda)$ 转到 $(pq|rs)$ 是 $O(N^5)$ 量级的计算；这对于 $(\mu \nu | \kappa \lambda)$ 转到 $(pp|qq)$ 是否也成立？
      
        提示：你可以借助 `np.einsum_path` 来辅助验证你的结论．
      
        提示：在程序实现过程中，你会用几维张量表示 $(ii|jj)^{A_t}$？六维还是四维？
      
2. 你可能注意到，我们使用 `scf_grad.grad_elec()` 而非 `scf_grad.de` 来验证结果．请查看 `scf_grad.grad_elec()` 与 `scf_grad.grad_nuc()` 的输出，并指出它们与 `scf_grad.de` 之间的关系．同时，需要注意到 `scf_grad.grad_elec()` 的调用是耗时的，即电子态能量的梯度并不预先存在梯度类中．

</div>

### 电子态能量的二阶梯度 $\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}$

In [ ]:
scf_hess.hess_elec.__func__

Yamaguchi (p428, V.2)；需要利用 Yamaguchi (p405, L.4-5) 的结论

\begin{align}
\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}
&= 2 h_{ii}^{A_t B_s} + 2 (ii|jj)^{A_t B_s} - (ij|ij)^{A_t B_s} - 2 \xi_{ii}^{A_t B_s} \varepsilon_i \\
&\quad + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p + 4 U_{pi}^{A_t} A_{pi, qj} U_{qj}^{B_s}
\end{align}

我们只将其中 $h_{ii}^{A_t B_s}, (ii|jj)^{A_t B_s}, (ij|ij)^{A_t B_s}$ 用 AO 基组表示出来，其余保留原样：

In [ ]:
np.allclose(
    + np.einsum("ABtsuv, uv -> ABts", H_2_ao, D)
    + 0.5 * np.einsum("ABtsuvkl, uv, kl -> ABts", eri2_ao, D, D)
    - 0.25 * np.einsum("ABtsukvl, uv, kl -> ABts", eri2_ao, D, D)
    - 2 * np.einsum("ABtsi, i -> ABts", Xi_2.diagonal(0, 4, 5)[:, :, :, :, so], eo)
    + 4 * np.einsum("Bspi, Atpi -> ABts", U_1[:, :, :, so], F_1_mo[:, :, :, so])
    + 4 * np.einsum("Atpi, Bspi -> ABts", U_1[:, :, :, so], F_1_mo[:, :, :, so])
    + 4 * np.einsum("Atpi, Bspi, p -> ABts", U_1[:, :, :, so], U_1[:, :, :, so], e)
    + 4 * np.einsum("Atpi, Bspi -> ABts", U_1[:, :, :, so], Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so])),
    scf_hess.hess_elec()
)

### 原子核互斥能的一阶梯度 $\frac{\partial}{\partial A_t} E_\mathrm{nuc}$

In [ ]:
scf_grad.grad_nuc.__func__

在计算原子核互斥能时，我们需要先定义一些方便的记号：

\begin{align}
Z_{MN} &= Z_M Z_N \\
V_{MNt} &= M_t - N_t \\
r_{MN} &= | \boldsymbol{M} - \boldsymbol{N} |
\end{align}

他们对应到程序分别是 ($r_{MN}$ 在程序实现中使用 $1/r_{MN}$)

In [ ]:
nuc_Z = np.einsum("M, N -> MN", mol.atom_charges(), mol.atom_charges())
nuc_V = lib.direct_sum("Mt - Nt -> MNt", mol.atom_coords(), mol.atom_coords())
nuc_rinv = 1 / (np.linalg.norm(nuc_V, axis=2) + np.diag([np.inf] * natm))

由此，一阶原子核互斥能可以表示为

$$
\frac{\partial}{\partial A_t} E_\mathrm{nuc} = - \frac{Z_{AM}}{r_{AM}^3} V_{AMt}
$$

In [ ]:
np.allclose(
    - np.einsum("AM, AM, AMt -> At", nuc_Z, nuc_rinv**3, nuc_V),
    scf_grad.grad_nuc()
)

### 原子核互斥能的二阶梯度 $\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{nuc}$

In [ ]:
scf_hess.hess_nuc.__func__

二阶核互斥能的梯度可以表示为

$$
\frac{\partial^2 E_\mathrm{nuc}}{\partial_{A_t} \partial_{B_s}} =
- 3 \frac{Z_{AB}}{r_{AB}^5} V_{ABt} V_{ABs}
+ 3 \frac{Z_{AM}}{r_{AM}^5} V_{AMt} V_{AMs}
+ \frac{Z_{AB}}{r_{AB}^3}
- \frac{Z_{AM}}{r_{AM}^3}
$$

为了写二阶梯度程序的便利，我们定义了一些稀疏的单位矩阵．

In [ ]:
mask_atm = np.eye(natm)[:, :, None, None]
mask_3D = np.eye(3)[None, None, :, :]

In [ ]:
hess_nuc = np.zeros((natm, natm, 3, 3))
hess_nuc += - 3 * np.einsum("AB, AB, ABt, ABs -> ABts", nuc_Z, nuc_rinv ** 5, nuc_V, nuc_V)
hess_nuc +=   3 * np.einsum("AM, AM, AMt, AMs -> Ats", nuc_Z, nuc_rinv ** 5, nuc_V, nuc_V)  * mask_atm
hess_nuc +=   np.einsum("AB, AB -> AB", nuc_Z, nuc_rinv ** 3)[:, :, None, None]             * mask_3D
hess_nuc += - np.einsum("AM, AM -> A", nuc_Z, nuc_rinv ** 3)[:, None, None, None]           * mask_atm * mask_3D

In [ ]:
np.allclose(
    hess_nuc,
    scf_hess.hess_nuc()
)

<div class="alert alert-info">

**任务**

1. (可选) 上面的代码利用到一些高级的 numpy 的 Boardcasting 技巧．尝试理解上述代码．上述代码的可读性未必比使用多个 for 语句来得好；你也可以尝试用 for 语句来生成二阶核互斥能的梯度．

2. (可选) 推导核互斥能公式．

3. (可选) 仿照 [一阶梯度](grad_rhf.ipynb#PySCF-一阶梯度计算) 与 [二阶梯度](grad_rhf.ipynb#PySCF-二阶梯度计算) 的数值求导程序，请尝试自己写一段小程序，验证核互斥能的一阶、二阶解析梯度与数值梯度是否一致．

   进阶：由于我们定义了便利的 HFHelper 类，我们还可以使用 lambda 函数，一行内解决数值梯度求取问题．以一阶数值与解析梯度的对照来举例子：
   
   ```
np.allclose(
    NumericDiff(mol, lambda mol : HFHelper(mol).scf_eng.energy_nuc()).get_numdif(),
    scf_grad.grad_nuc()
)
   ```
   
   试仿照上述代码，写出核互斥能二阶解析与数值梯度相互验证的代码．

</div>

## 梯度量的补充定义

### 通用记号与程序变量

在继续叙述之前，我们需要对一些梯度记号进行定义：

<div class="alert alert-info">

**记号定义**

* 偏导记号 $\frac{\partial}{\partial A_t} = \partial_{A_t} + \partial_{A_t}^\mathrm{U}$ 与一般的偏导记号不太相同．在以后，我们会将所有 Skeleton 导数记为 $\partial_{A_t}$，而将产生 U 矩阵的导数记为 $\partial_{A_t}^\mathrm{U}$．这样的记号体系可能需要在以后习惯一下，因为这不是很正规的记号．

* 对于重叠矩阵 $S_{\mu \nu}$、Hamiltonian Core 矩阵 $h_{\mu \nu}$、Fock 矩阵 $F_{\mu \nu}$、ERI 积分 $(\mu \nu | \kappa \lambda)$，包括四维算符 $A_{pqrs}$，上标了原子核坐标分量记号的矩阵对应的是 Skeleton 导数；举例来说即 $F_{\mu \nu}^{A_t B_s} = \partial_{A_t} \partial_{B_s} F_{\mu \nu}$．这一般不意味着 $\frac{\partial}{\partial A_t} F_{\mu \nu} = F_{\mu \nu}^{A_t B_s}$．

* 对于 $U_{pq}, B_{pq}, \xi_{pq}$ 等矩阵而言，其上标的原子核坐标分量没有像上面一样明确的意义．每个这样的记号都需要作各自的定义；即使我们也许会说 $U_{pq}^{A_t B_s}$ 显然是二阶 U 矩阵，但我们不应该说这是一阶 U 矩阵的 Skeleton 导数矩阵．

* 原子轨道角标的下标若有原子记号，举例来说对于 $D_{\mu_A \nu_B}$，就表示为密度矩阵只有当 $\mu$ 是 A 原子的 AO 轨道，而 $\nu$ 是 B 原子的 AO 轨道时，我们才考虑 $D_{\mu \nu}$ 的值，否则为零

    * 如果遇到双下标的情况，即 $D_{\mu_{AB} \nu}$，那么就意味着只有当 $\mu$ 是 A 且 B 的 AO 轨道时才考虑密度矩阵的值；这隐含了 $A \neq B$ 时，我们就不考虑其贡献值．

    * 记号的主要用途是处理以原子为基础的分割 (slice) 时，方便书写程序所定义的．对于这个记号，我们可以简单地理解为 $D_{\mu_A \nu_B} = D_{\mu \nu} \cdot \delta_{\mu \in A} \delta_{\nu \in B}$．因此，$D_{\mu_A \nu_B} \langle \mu_{At} | \nu_{Bs} \rangle$ 与 $D_{\mu_A \nu_B} \langle \mu_t | \nu_s \rangle = D_{\mu_A \nu_B} \langle \partial_t \mu | \partial_s \nu \rangle$ 具有相同的意义．

</div>

同时，我们会把以后经常需要使用的电子积分在这里预先作定义；这些电子积分都只是与分子有关，而与自洽场密度无关的张量；我们同时定义一个便利函数 `mol_slice`：

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! NOTE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For clear and simplicity, following code cell will be hidden and assumed to be executed!

# 1-deriv
int1e_ipovlp = mol.intor("int1e_ipovlp")
int1e_ipkin  = mol.intor("int1e_ipkin" )
int1e_ipnuc  = mol.intor("int1e_ipnuc" )
int2e_ip1    = mol.intor("int2e_ip1"   )

# 2-deriv
int1e_ipipkin  = mol.intor("int1e_ipipkin" ).reshape(3, 3, nao, nao)
int1e_ipkinip  = mol.intor("int1e_ipkinip" ).reshape(3, 3, nao, nao)
int1e_ipipnuc  = mol.intor("int1e_ipipnuc" ).reshape(3, 3, nao, nao)
int1e_ipnucip  = mol.intor("int1e_ipnucip" ).reshape(3, 3, nao, nao)
int1e_ipipovlp = mol.intor("int1e_ipipovlp").reshape(3, 3, nao, nao)
int1e_ipovlpip = mol.intor("int1e_ipovlpip").reshape(3, 3, nao, nao)
int2e_ipip1    = mol.intor("int2e_ipip1"   ).reshape(3, 3, nao, nao, nao, nao)
int2e_ipvip1   = mol.intor("int2e_ipvip1"  ).reshape(3, 3, nao, nao, nao, nao)
int2e_ip1ip2   = mol.intor("int2e_ip1ip2"  ).reshape(3, 3, nao, nao, nao, nao)

def mol_slice(atm_id, mol=mol):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    return slice(p0, p1)

下面每一行对变量的说明中，包含变量名、变量维度与角标意义、以及其具体的公式形式．

电子积分一阶导数

* `int1e_ipovlp`, $t \mu \nu$: $\langle \partial_t \mu | \nu \rangle$

* `int1e_ipkin`, $t \mu \nu$: $\langle \partial_t \mu | \hat t | \nu \rangle$

* `int1e_ipnuc`, $t \mu \nu$: $\langle \partial_t \mu | \hat v_\mathrm{nuc} | \nu \rangle$

* `int2e_ip1`, $t \mu \nu \kappa \tau$: $\langle \partial_t \mu \nu | \kappa \lambda \rangle$

电子积分二阶导数

* `int1e_ipipkin`, $t s \mu \nu$ : $\langle \partial_t \partial_s \mu | \hat t | \nu \rangle$

* `int1e_ipkinip`, $t s \mu \nu$ : $\langle \partial_t \mu | \hat t | \partial_s \nu \rangle$

* `int1e_ipipnuc`, $t s \mu \nu$ : $\langle \partial_t \partial_s \mu | \hat v_\mathrm{nuc} | \nu \rangle$

* `int1e_ipnucip`, $t s \mu \nu$ : $\langle \partial_t \mu | \hat v_\mathrm{nuc} | \partial_s \nu \rangle$

* `int1e_ipipovlp`, $t s \mu \nu$ : $(\partial_t \partial_s \mu | \nu)$

* `int1e_ipovlpip`, $t s \mu \nu$ : $(\partial_t \mu | \partial_s \nu)$

* `int2e_ipip1`, $t s \mu \nu \kappa \tau$ : $(\partial_t \partial_s \mu \nu | \kappa \lambda)$

* `int2e_ipvip1`, $t s \mu \nu \kappa \tau$ : $(\partial_t \mu \partial_s \nu | \kappa \lambda)$

* `int2e_ip1ip2`, $t s \mu \nu \kappa \tau$ : $(\partial_t \mu \nu | \partial_s \kappa \lambda)$

函数 `mol_slice` 传入原子序号，返回 AO 轨道中属于该原子的分割．

注意到我们并没有把与 $\langle \mu | \frac{1}{| \boldsymbol{r} |} | \nu \rangle$ 有关的核排斥势积分相关的定义放在这里．这是因为和排斥势通常需要现用现生成，在 [RHF 笔记](basic_scf.ipynb#核排斥势积分) 中实际上我们已经有所提及．

<div class="alert alert-warning">

**注意**

以后的笔记中，上述代码块将假定被执行过并会隐藏．

</div>

### 求取梯度范例：$S_{\mu \nu}^{A_t}$

尽管这份笔记中，大多数内容都是为了程序化而服务；但我们的程序化的目标并不是单纯地调用 PySCF 的高层函数以计算非自洽梯度；我们还需要了解一些底层的电子积分、DFT 导数的计算方法．这是出于以下的考量 (即使也许不切实际)：

* 如果以后 PySCF 的高层函数调用方式发生了变化，即使我们的代码可能效率不高但仍然可以用底层的方法作实现；

* 如果以后我们打算从头写自己的代码，这部分的学习应当很有帮助；

* 退一万步，如果 PySCF 不再开源，我们需要拿出方案来在别的软件中实现；即使不同软件里高层函数的目的不同，但一些必须计算的底层实现一般应是相同的．

* HFHelper 类实际上也是我自己写的．谁又能保证一个二年级研究生能把代码写对呢？另一个等价的问题是，PySCF 实际上不是一个拥有很充分注释的软件，我们需要怎样验证 PySCF 的高层函数所生成的张量与它表面上的方法或成员名称相同？

为此，我们不应当止步于在已有 Skeleton 导数矩阵的情况下推导梯度结果，而应当进一步问，这些 Skeleton 导数矩阵是怎么得到的．我们把推导的终结定在从分子本身出发构建的电子积分张量；如何求取电子积分不是依靠 Python 能解决的问题，并且这至少还要涉及一些更为复杂的原子轨道与基组之间关系、以及作为 C++ 程序的电子积分库要如何调用的问题，因此，我们不考虑电子积分本身是如何获取的了．

我们先拿最简单的例子来作说明．以后的推导中，很多步骤可能不像这里一样细致了．

重叠积分的 Skeleton 导数是

$$
S_{\mu \nu}^{A_t} = \partial_{A_t} S_{\mu \nu} = \partial_{A_t} \langle \mu | \nu \rangle = \langle \partial_{A_t} \mu | \nu \rangle + \langle \mu | \partial_{A_t} \nu \rangle
$$

我们要将上述的原子核坐标导数化为电子坐标导数；因为 PySCF 程序中只会给出电子坐标分量导数的积分张量．

我们可能会在以后，出于简化公式，或者简化代码的目的，将上式写为

$$
S_{\mu \nu}^{A_t} = \langle \partial_{A_t} \mu | \nu \rangle + \mathrm{interchange} (\mu, \nu)
$$

尽管在这样一个简单的例子里，多写一个 $\mathrm{interchange}$ 似乎是更累赘．

随后我们要了解如何求取梯度积分 $\langle \partial_{A_t} \mu | \nu \rangle$．在这一段我们暂时不用简化的记号．

$$
\langle \partial_{A_t} \mu | \nu \rangle = \int \partial_{A_t} \phi_\mu (\boldsymbol{r} - \boldsymbol{M}^\mu) \cdot \phi_\nu (\boldsymbol{r} - \boldsymbol{M}^\nu) \, \mathrm{d} \boldsymbol{r}
$$

上式中的 $\boldsymbol{M}^\mu$ 指的是 $\mu$ 原子轨道所属的原子的三维坐标．尽管大多数时候我们只要把原子轨道当作电子的函数即可，但核坐标梯度导数却是比较少有的必须把原子核位置写出的情况之一．与之比较相近的记号在 Szabo (p153, 3.202)．

现在我们来看 $\partial_{A_t} \phi_\mu (\boldsymbol{r} - \boldsymbol{M}^\mu)$．如果 $\mu$ 所指向的原子不是 $A$，那么 $\phi_\mu (\boldsymbol{r} - \boldsymbol{M}^\mu)$ 中就没有任何与 $A_t$ 有关的变量了；因此其贡献为零．但若 $\mu$ 所指向的原子是 $A$，那么 $\boldsymbol{M}^\mu$ 从定义上即 $\boldsymbol{A}$．

现在我们来看 $\partial_{A_t} \phi_{\mu_A} (\boldsymbol{r} - \boldsymbol{A})$．$\partial_{A_t}$ 是原子坐标 $\boldsymbol{A}$ 三个维度中的一个分量；现在假设，如果 $A_t$ 是原子 $A$ 的核坐标 $\boldsymbol{A} = (A_x, A_y, A_z)$ 的 $A_x$ 方向分量，那么我们记 $t$ 代表电子坐标 $\boldsymbol{r} = (x, y, z)$ 的分量 $x$．那么这个求导问题与分量 $A_y, A_z$ 原子坐标分量和 $y, z$ 电子坐标分量就无关了．因此，上述求导问题化为 (若 $A_t := A_x$)

$$
\partial_{A_t} \phi_{\mu_A} (\boldsymbol{r} - \boldsymbol{A}) \Big|_{A_t = A_x} = \partial_{A_x} \phi_{\mu_A} (x - A_x)
$$

我们临时定义一个变量 $u = x - A_x$ (这个为了方便定义的变量以后不会出现)，那么由于电子坐标与原子坐标本身是毫无关系的，因此 $\partial_{A_x} u = -1$，而 $\partial_x u = 1$．同时，我们根据偏导法则，可知

$$
\partial_{A_x} \phi_{\mu_A} (u) = \partial_u \phi_{\mu_A} (u) \cdot \partial_{A_x} u = - \partial_u \phi_{\mu_A} (u)
$$

以及

$$
\partial_{x} \phi_{\mu_A} (u) = \partial_u \phi_{\mu_A} (u) \cdot \partial_{x} u = \partial_u \phi_{\mu_A} (u)
$$

因此，

$$
\partial_{A_x} \phi_{\mu_A} (u) = - \partial_{x} \phi_{\mu_A} (u)
$$

这样就把原子核坐标分量导数的问题成功地化解为电子和坐标分量的导数问题了．这对于 $A_t := A_y$ 或 $A_t := A_z$ 的情形是同样的．

回顾我们在讲述 [动能积分](basic_scf.ipynb#动能积分) 时，我们把 $\partial_t \phi_{\mu}$ 记作 $\phi_{t \mu}$．对于 $\mu$ 下有角标 $A$ 的情况，我们一样地记为 $\phi_{t \mu_A}$．因此，现在我们把记号渐渐重新化繁为简 (当然多少也渐渐地不正规起来)

$$
\langle \partial_{A_t} \mu | \nu \rangle = \partial_{A_t} \phi_\mu \cdot \phi_\nu = - \partial_t \phi_{\mu_A} \cdot \phi_\nu = - \phi_{t \mu_A} \cdot \phi_\nu = - \langle \mu_{At} | \nu \rangle
$$

注意到上面从第二个等号开始，$t$ 与 $A$ 之间的关系不再是上下标的关系了；这意味着 $t$ 的意义从原子坐标分量转为电子坐标分量，因此不再受原子核的控制了．在变为电子坐标分量的过程中，由于经过一步 $\partial_{A_t}^\mathrm{U}$ 的偏导数，从而产生出一个负号．这个关系式在以后会非常经常地使用．

从而，$S_{\mu \nu}^{A_t}$ 可以记为

$$
S_{\mu \nu}^{A_t} = - \langle \mu_{At} | \nu \rangle + \mathrm{interchange} (\mu, \nu)
$$

现在我们通过程序构建 $S_{\mu \nu}^{A_t}$ 的张量．我们可以在构建张量的过程中使用循环，以允许在某个原子下，只拿出该原子所在的原子轨道．我们以尾置下划线区分我们的实现与 HFHelper 的实现．

In [ ]:
S_1_ao_ = np.zeros((natm, 3, nao, nao))
for A in range(natm):
    sA = mol_slice(A)
    S_1_ao_[A, :, sA, :] += - int1e_ipovlp[:, sA, :]
S_1_ao_ += S_1_ao_.transpose(0, 1, 3, 2)
np.allclose(S_1_ao_, S_1_ao)

通过上述代码，我们一方面验证了我们上面公式推导的合理性；另一方面，我们也了解了 HFHelper 所生成的 `S_1_ao` 确实应当是我们想象的 $S_{\mu \nu}^{A_t}$．我们也许会说这是一个双向验证的过程；但我们也会说也许两边的结果都是错的，只是凑巧 `np.allclose` 给出了 True 的结果．

事实上，我们会说，由于 $S_{\mu \nu}$ 不存在分子轨道的贡献，因此 $\partial_{A_t}^\mathrm{U} S_{\mu \nu} = 0$；因此，

$$
\frac{\partial}{\partial_{A_t}} S_{\mu \nu} = (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) S_{\mu \nu} = \partial_{A_t} S_{\mu \nu} + \partial_{A_t}^\mathrm{U} S_{\mu \nu} = S_{\mu \nu}^{A_t} + 0 = S_{\mu \nu}^{A_t}
$$

$\frac{\partial}{\partial_{A_t}} S_{\mu \nu}$ 其实代表了 $S_{\mu \nu}$ 对 $A_t$ 的完整偏导数，因此我们还可以使用数值梯度来实现 $S_{\mu \nu}^{A_t}$．在假定 $S_{\mu \nu}$ 生成正确的情况下，我们可以验证 HFHelper 中 $S_{\mu \nu}^{A_t}$ 是否基本正确．这里我们通过 `mol.intor` 的方法来生成 $S_{\mu \nu}$，可以参考 [小型自洽场程序](basic_scf.ipynb#小型自洽场程序)．

In [ ]:
np.allclose(
    NumericDiff(mol, lambda mol : mol.intor("int1e_ovlp")).get_numdif(),
    S_1_ao
)

## 实现参考：一阶 Skeleton 导数

### Hamiltonian Core 积分一阶 Skeleton 导数 $h_{\mu \nu}^{A_t}$

Hamiltonian Core 在计算过程中可能要分为几部分相加．我们将这几部分的编号放在右上标．

\begin{align}
h_{\mu \nu}^{A_t} &= \langle \partial_{A_t} \mu | \hat t | \nu \rangle + \langle \partial_{A_t} \mu | \hat v_\mathrm{nuc} | \nu \rangle + \frac{1}{2} \langle \mu | \partial_{A_t} \hat v_\mathrm{nuc} | \nu \rangle \\
&\quad + \mathrm{interchange} (\mu, \nu)
\tag{H1.1} \label{eq.H1.1}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t, 1} := \langle \partial_{A_t} \mu | \hat t | \nu \rangle = - \langle \mu_{At} | \hat t | \nu \rangle
\tag{H1.2} \label{eq.H1.2}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t, 2} := \langle \partial_{A_t} \mu | \hat v_\mathrm{nuc} | \nu \rangle = - \langle \mu_{At} | \hat v_\mathrm{nuc} | \nu \rangle
\tag{H1.3} \label{eq.H1.3}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t, 3}
&:= \frac{1}{2} \langle \mu | \partial_{A_t} \hat v_\mathrm{nuc} | \nu \rangle \\
&= \frac{1}{2} \langle \mu | \partial_{A_t} \frac{-Z_M}{|\boldsymbol{r} - \boldsymbol{M}|} | \nu \rangle
= \frac{1}{2} \langle \mu | \partial_{A_t} \frac{-Z_A}{|\boldsymbol{r} - \boldsymbol{A}|} | \nu \rangle \\
&= \frac{1}{2} \langle \partial_{A_t} \frac{-Z_A}{|\boldsymbol{r}|} | \mu \nu \rangle_{\boldsymbol{r} \rightarrow A}
= \frac{1}{2} \langle - \partial_{t} \frac{-Z_A}{|\boldsymbol{r}|} | \mu \nu \rangle_{\boldsymbol{r} \rightarrow A} \\
&= \frac{1}{2} \langle \frac{-Z_A}{|\boldsymbol{r}|} | \partial_{t} (\mu \nu) \rangle_{\boldsymbol{r} \rightarrow A} \\
&= \frac{1}{2} \langle \mu_t | \frac{-Z_A}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow A}
+ \mathrm{interchange} (\mu, \nu)
\tag{H1.4} \label{eq.H1.4}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t} &= h_{\mu \nu}^{A_t, 1} + h_{\mu \nu}^{A_t, 2} + h_{\mu \nu}^{A_t, 3} + \mathrm{interchange} (\mu, \nu) \\
&= - \langle \mu_{At} | \hat t | \nu \rangle - \langle \mu_{At} | \hat v_\mathrm{nuc} | \nu \rangle + \langle \mu_t | \frac{-Z_A}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow A} \\
&\quad + \mathrm{interchange} (\mu, \nu)
\tag{H1.5} \label{eq.H1.5}
\end{align}

In [ ]:
H_1_ao_ = np.zeros((natm, 3, nao, nao))
for A in range(natm):
    sA = mol_slice(A)
    H_1_ao_[A, :, sA, :] += - int1e_ipkin[:, sA, :]
    H_1_ao_[A, :, sA, :] += - int1e_ipnuc[:, sA, :]
    with mol.with_rinv_as_nucleus(A):
        H_1_ao_[A] += - mol.atom_charge(A) * mol.intor("int1e_iprinv")
H_1_ao_ += H_1_ao_.transpose(0, 1, 3, 2)
np.allclose(H_1_ao_, H_1_ao)

与 $S_{\mu \nu}^{A_t}$ 一样，$\partial_{A_t}^\mathrm{U} h_{\mu \nu} = 0$；因此，对矩阵 $h_{\mu \nu}$ 的数值导数也可以得到 $h_{\mu \nu}^{A_t}$：

In [ ]:
np.allclose(
    NumericDiff(mol, lambda mol : HFHelper(mol).H_0_ao).get_numdif(),
    H_1_ao
)

同时，在 PySCF 中，有高级函数专门用于生成 $h_{\mu \nu}^{A_t}$：

In [ ]:
scf_grad.hcore_generator.__func__

In [ ]:
np.allclose(
    [scf_grad.hcore_generator()(A) for A in range(natm)],
    H_1_ao
)

<div class="alert alert-info">

**任务**

1. 这一段推导中，(H1.2) 与 (H1.3) 的推导与 $S_{\mu \nu}^{A_t}$ 的推导几乎是一致的；但 (H1.4) 的推导则稍有不同．你需要对每个等号成立的合理性进行思考，并回答下面的问题：

    1. 该等式中一共出现了 7 个等号；第 1 个等号是定义式．如果你对第 2 个等号有疑问，请先翻阅 RHF 笔记中有关 [核排斥势](basic_scf.ipynb#核排斥势积分) 的描述，熟悉记号体系．
   
    2. 如果你对第 3, 5, 7 个等号的左右有疑问，请重新理解 $S_{\mu \nu}^{A_t}$ 的生成过程，并重新理解 (H1.2) 与 (H1.3) 的推导．如果你对 (H1.1) 的等式左右还有疑问，请先翻看一遍 Yamaguchi 书本的第 3, 4, 10 章．这份笔记已经假定你有这三章的基础．
   
    3. 如果你对第 4 个等号有疑问，请仍然翻阅 RHF 中 [核排斥势](basic_scf.ipynb#核排斥势积分) 的描述．这一步的意义是，由于 PySCF 的积分引擎只提供 $1/|\boldsymbol{r}|$ 的积分，因此我们需要手动平移分子的坐标．临时地平移分子坐标在 PySCF 的 `mol.with_rinv_as_nucleus` 函数可以实现，但它只在进行 `int1e_iprinv` 积分时才会体现出坐标的平移；你无法通过输出 `mol.atom_coords` 查看平移后的分子坐标．
   
    4. 如果你对第 6 个等号有疑问，请考虑你在 RHF 中 [动能积分](basic_scf.ipynb#动能积分) 的任务 2 的回答．我的理解是 $\partial_t$ 作为电子坐标分量偏导算符，是反厄米算符；同时，$1/|\boldsymbol{r}|$ 作为和排斥势算符的一部分，它可以处在被积函数的任意位置，不似偏导算符．因此，等号 6 成立．
   
    5. 在 (H1.4) 中，积分前有系数 $1/2$；为何到 (H1.5) 就消失了？
   
2. 似乎我们已经检查了所有等号的合理性了；上面的公式确实正确无误了．但是在我自己推导这些公式的过程中，曾经还有其它的疑问．下面的问题沿着思考深度递进：

    1. 为何我们需要用到反厄米性？有没有简单的办法得到 $\partial_t \frac{1}{|\boldsymbol{r}|}$？
   
    2. $\boldsymbol{r} \rightarrow A$ 的记号看起来很不友好，我自己似乎很想避免使用这个记号．因此我曾经在思考，对于 (H1.4) 的第三个等式左，能否给出以下等价关系？
   
        $$
        \frac{1}{2} \langle \mu | \partial_{A_t} \frac{-Z_M}{|\boldsymbol{r} - \boldsymbol{M}|} | \nu \rangle
        \quad ? \quad - \frac{1}{2} \langle \frac{-Z_M}{|\boldsymbol{r} - \boldsymbol{M}|} | \partial_{A_t} (\mu \nu) \rangle
        $$
      
        似乎这样就可以规避 $\boldsymbol{r} \rightarrow A$ 的使用．但事实上我们不能这样做．为什么上面的等式不成立？
   
    3. 进一步地，从以前 $S_{\mu \nu}$ 的推导中，我们知道大多数时候 $\partial_{A_t}$ 算符可以看作 $- \partial_t$；那么是否 $\partial_{A_t}$ 也与 $- \partial_t$ 一样具有反厄米性？
   
    4. $h_{\mu \nu}^{A_t, 1}$ 与 $h_{\mu \nu}^{A_t, 2}$ 的电子积分贡献项似乎都有 $\mu_{At}$ 的形式，表明只有 $\mu$ 是原子 $A$ 的轨道时，表达式才有意义；为何在 $h_{\mu \nu}^{A_t, 3}$ 的最终结果中，我们没有看到 $\mu_{At}$ 而只看到 $\mu_t$？

</div>

### Fock 矩阵一阶 Skeleton 导数 $F_{\mu \nu}^{A_t}$

Yamaguchi (p407, M.4)

$$
F_{\mu \nu}^{A_t} = h_{\mu \nu}^{A_t} + (\mu \nu | \kappa \lambda)^{A_t} D_{\kappa \lambda} - \frac{1}{2} (\mu \kappa | \nu \lambda)^{A_t} D_{\kappa \lambda}
$$

In [ ]:
F_1_ao_ = (
    + H_1_ao
    + np.einsum("Atuvkl, kl -> Atuv", eri1_ao, D)
    - 0.5 * np.einsum("Atukvl, kl -> Atuv", eri1_ao, D)
)
np.allclose(F_1_ao, F_1_ao_)

在 PySCF 中，也有函数专门用于生成 $F_{\mu \nu}^{A_t}$，但它位于 Hessian 求取的程序中：

In [ ]:
scf_hess.make_h1.__func__

In [ ]:
np.allclose(
    scf_hess.make_h1(mo_coeff=C, mo_occ=mo_occ),
    F_1_ao
)

$(\mu \nu | \kappa \lambda)^{A_t}$ 的构造在后面一小节叙述．

<div class="alert alert-info">

**任务**

1. 我们以前可以用数值梯度来直接验证 $S_{\mu \nu}^{A_t}$ 与 $h_{\mu \nu}^{A_t}$ 的构造是否正确；试问对于 $F_{\mu \nu}^{A_t}$ 是否也能这么做？为什么？

</div>

### ERI 一阶导数 $(\mu \nu | \kappa \lambda)^{A_t}$

\begin{align}
(\mu \nu | \kappa \lambda)^{A_t}
&= (\partial_{A_t} \mu \nu | \kappa \lambda) + (\mu \partial_{A_t} \nu | \kappa \lambda) + (\mu \nu | \partial_{A_t} \kappa \lambda) + (\mu \nu | \kappa \partial_{A_t} \lambda) \\
&= - (\mu_{At} \nu | \kappa \lambda) - (\mu \nu_{At} | \kappa \lambda) - (\mu \nu | \kappa_{At} \lambda) - (\mu \nu | \kappa \lambda_{At})
\end{align}

In [ ]:
eri1_ao_ = np.zeros((natm, 3, nao, nao, nao, nao))
for A in range(natm):
    sA = mol_slice(A)
    eri1_ao_[A, :, sA, :, :, :] -= int2e_ip1[:, sA]
    eri1_ao_[A, :, :, sA, :, :] -= int2e_ip1[:, sA].transpose(0, 2, 1, 3, 4)
    eri1_ao_[A, :, :, :, sA, :] -= int2e_ip1[:, sA].transpose(0, 3, 4, 1, 2)
    eri1_ao_[A, :, :, :, :, sA] -= int2e_ip1[:, sA].transpose(0, 3, 4, 2, 1)
np.allclose(eri1_ao_, eri1_ao)

<div class="alert alert-info">

**任务**

1. 试问对于 $(\mu \nu | \kappa \lambda)^{A_t}$ 能否用 $(\mu \nu | \kappa \lambda)$ 的数值梯度验证？

    提示：也许你会发现使用 `np.allclose` 后，结果是 False．你可以尝试输出这个矩阵，并适当增大判断阈值；这个阈值应当是你认为可以接受的范围．以后也许会经常遇到这种情况．在这个例子中，设定 `atol=1e-7` 或者 `rtol=1e-4` 可以输出 True．

</div>

## 实现参考：二阶 Skeleton 导数

### Hamiltonian Core 积分二阶 Skeleton 导数 $h_{\mu \nu}^{A_t B_s}$

二阶 Hamiltonian Core 导数会变得相当复杂．

\begin{align}
h_{\mu \nu}^{A_t B_s}
&= \langle \partial_{A_t} \partial_{B_s} \mu | \hat t | \nu \rangle + \langle \partial_{A_t} \partial_{B_s} \mu | \hat v_\mathrm{nuc} | \nu \rangle \\
&\quad + \langle \partial_{A_t} \mu | \hat t | \partial_{B_s} \nu \rangle + \langle \partial_{A_t} \mu | \hat v_\mathrm{nuc} | \partial_{B_s} \nu \rangle \\
&\quad + \langle \partial_{A_t} \mu | \partial_{B_s} \hat v_\mathrm{nuc} | \nu \rangle + \langle \partial_{B_s} \mu | \partial_{A_t} \hat v_\mathrm{nuc} | \nu \rangle \\
&\quad + \frac{1}{2} \langle \mu | \partial_{A_t} \partial_{B_s} \hat v_\mathrm{nuc} | \nu \rangle \\
&\quad + \mathrm{interchange} (\mu, \nu)
\tag{H2.1} \label{eq.H2.1}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t B_s, 1} &:= \langle \partial_{A_t} \partial_{B_s} \mu | \hat t | \nu \rangle + \langle \partial_{A_t} \partial_{B_s} \mu | \hat v_\mathrm{nuc} | \nu \rangle \\
&= \langle \mu_{ABts} | \hat t | \nu \rangle + \langle \mu_{ABts} | \hat v_\mathrm{nuc} | \nu \rangle
\tag{H2.2} \label{eq.H2.2}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t B_s, 2} &:= \langle \partial_{A_t} \mu | \hat t | \partial_{B_s} \nu \rangle + \langle \partial_{A_t} \mu | \hat v_\mathrm{nuc} | \partial_{B_s} \nu \rangle \\
&= \langle \mu_{At} | \hat t | \nu_{Bs} \rangle + \langle \mu_{At} | \hat v_\mathrm{nuc} | \nu_{Bs} \rangle
\tag{H2.3} \label{eq.H2.3}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t B_s, 3} &:= \langle \partial_{A_t} \mu | \partial_{B_s} \hat v_\mathrm{nuc} | \nu \rangle + \langle \partial_{B_s} \mu | \partial_{A_t} \hat v_\mathrm{nuc} | \nu \rangle \\
&= - \langle \mu_{Ats} | \frac{-Z_B}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow B}
- \langle \mu_{At} | \frac{-Z_B}{|\boldsymbol{r}|} | \nu_s \rangle_{\boldsymbol{r} \rightarrow B} \\
&\quad
- \langle \mu_{Bts} | \frac{-Z_A}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow A}
- \langle \mu_{Bs} | \frac{-Z_A}{|\boldsymbol{r}|} | \nu_t \rangle_{\boldsymbol{r} \rightarrow A}
\tag{H2.4} \label{eq.H2.4}
\end{align}

\begin{align}
h_{\mu \nu}^{A_t B_s, 4} &:= \frac{1}{2} \langle \mu | \partial_{A_t} \partial_{B_s} \hat v_\mathrm{nuc} | \nu \rangle \\
&= \frac{1}{2} \langle \mu | \partial_{t} \partial_{s} \frac{-Z_A}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow AB} \\
&= \frac{1}{2} \langle \frac{-Z_A}{|\boldsymbol{r}|} | \partial_{t} \partial_{s} (\mu \nu) \rangle_{\boldsymbol{r} \rightarrow AB} \\
&= \frac{1}{2} \langle \mu_{ts} | \frac{-Z_A}{|\boldsymbol{r}|} | \nu \rangle_{\boldsymbol{r} \rightarrow AB} + \frac{1}{2} \langle \mu_t | \frac{-Z_A}{|\boldsymbol{r}|} | \nu_s \rangle_{\boldsymbol{r} \rightarrow AB} \\
&\quad + \mathrm{interchange} (\mu, \nu)
\tag{H2.5} \label{eq.H2.5}
\end{align}

由此，我们就可以以程序构建 $h_{\mu \nu}^{A_t}$ 了．

In [ ]:
H_2_ao_ = np.zeros((natm, natm, 3, 3, nao, nao))
for A in range(natm):
    sA = mol_slice(A)
    H_2_ao_[A, A, :, :, sA, :] += int1e_ipipkin[:, :, sA, :]  # contrib 1
    H_2_ao_[A, A, :, :, sA, :] += int1e_ipipnuc[:, :, sA, :]  # contrib 1
    with mol.with_rinv_as_nucleus(A):
        H_2_ao_[A, A] += - mol.atom_charge(A) * mol.intor("int1e_ipiprinv").reshape(3, 3, nao, nao)  # contrib 4
        H_2_ao_[A, A] += - mol.atom_charge(A) * mol.intor("int1e_iprinvip").reshape(3, 3, nao, nao)  # contrib 4
    for B in range(natm):
        sB = mol_slice(B)
        H_2_ao_[A, B, :, :, sA, sB] += int1e_ipkinip[:, :, sA, sB]  # contrib 1
        H_2_ao_[A, B, :, :, sA, sB] += int1e_ipnucip[:, :, sA, sB]  # contrib 2
        with mol.with_rinv_as_nucleus(B):
            H_2_ao_[A, B, :, :, sA, :] += mol.atom_charge(B) * mol.intor("int1e_ipiprinv").reshape(3, 3, nao, nao)[:, :, sA, :]  # contrib 3
            H_2_ao_[A, B, :, :, sA, :] += mol.atom_charge(B) * mol.intor("int1e_iprinvip").reshape(3, 3, nao, nao)[:, :, sA, :]  # contrib 3
        with mol.with_rinv_as_nucleus(A):
            H_2_ao_[A, B, :, :, sB, :] += mol.atom_charge(A) * mol.intor("int1e_ipiprinv").reshape(3, 3, nao, nao)[:, :, sB, :]  # contrib 3
            H_2_ao_[A, B, :, :, sB, :] += mol.atom_charge(A) * mol.intor("int1e_iprinvip").reshape(3, 3, nao, nao).swapaxes(0, 1)[:, :, sB, :]  # contrib 3
H_2_ao_ += H_2_ao_.transpose(0, 1, 2, 3, 5, 4)
np.allclose(H_2_ao_, H_2_ao)

同 $h_{\mu \nu}^{A_t} = \frac{\partial}{\partial {A_t}} h_{\mu \nu}$，关系式 $h_{\mu \nu}^{A_t B_s} = \frac{\partial}{\partial {B_s}} h_{\mu \nu}^{A_t}$ 同样成立：

In [ ]:
np.allclose(
    NumericDiff(mol, lambda mol : HFHelper(mol).get_H_1_ao(), deriv=2).get_numdif(),
    H_2_ao,
    atol=1e-7
)

PySCF 中，存在函数生成 $h_{\mu \nu}^{A_t B_s}$：

In [ ]:
scf_hess.hcore_generator.__func__

它传入两个原子序号，导出这两个原子对 $h_{\mu \nu}^{A_t B_s}$ 的贡献．其张量维度是 4 维；前 2 维度是三维坐标，后 2 维度是原子轨道数．

In [ ]:
np.allclose(
    [[scf_hess.hcore_generator()(A, B) for B in range(natm)] for A in range(natm)],
    H_2_ao
)

<div class="alert alert-info">

**任务**

1. 尝试验证导出 $h_{\mu, \nu}^{A_t B_s}$ 的关系式．其中，$\boldsymbol{r} \rightarrow AB$ 符号是指将 $A$ 原子的坐标设定为原点求电子积分，同时 $B$ 必须要与 $A$ 是等价原子．

2. 请尝试 `[[scf_hess.hcore_generator()(A, B) for A in range(natm)] for B in range(natm)]`，查看是否与 `H_2_ao` 相等．

    提示：请思考 [循环索引顺序的区别](python_intro.ipynb#循环)．
   
3. 对称性考察：

    1. 请考察 $h_{\mu \nu}^{A_t B_s}$ 张量的对称性．如果我们说 $h_{\mu \nu}$ 是二重对称的 ($h_{\mu \nu} = h_{\nu \mu}$)，而 $(\mu \nu | \kappa \lambda)$ 是八重对称的 (当 $\mu, \nu, \kappa, \lambda$ 全部不相同时，在 24 种可能轮换中存在 8 种不同的轮换，其 ERI 积分值相等)，那么 $h_{\mu \nu}^{A_t B_s}$ 是几重对称的？
   
    2. 为何在生成 `H_2_ao_` 的代码块中，$\langle \mu_{Bs} | \frac{Z_A}{|\boldsymbol{r}|} | \nu_t \rangle_{\boldsymbol{r} \rightarrow A}$ 的生成代码中有 `swapaxes` 出现？其作用是什么？

4. (可选) 上述生成 `H_2_ao_` 的代码不算很高效．你可以尝试至少以下方法提高代码效率：

    1. 利用 $h_{\mu \nu}^{A_t B_s}$ 中关于 $A_t, B_s$ 的对称性，应当可以降低代码运行时间约 1.5 倍；
   
    2. `mol.intor("int1e_ipiprinv")` 与 `mol.intor("int1e_iprinvip")` 命令并不是调用已经算完的电子积分，而是现算电子积分．我们也许会认为类似于 $h_{\mu \nu}^{A_t B_s}$ 张量在内存的储存消耗是微乎其微的；因此我们完全可以考虑预先生成这类张量．同时，我们会注意到类似于 $\langle \mu_{ABts} | \hat t | \nu \rangle$ 与 $\langle \mu_{ABts} | \hat v_\mathrm{nuc} | \nu \rangle$ 总是成对出现的，因此可以通过增加一次加法计算的代价减少一倍张量缩并的 CPU 消耗．通过这些技巧，应当可以对当前体系的计算效率提高 3 倍．

</div>

### 重叠积分二阶 Skeleton 导数 $S_{\mu \nu}^{A_t B_s}$

$$
S_{\mu \nu}^{A_t B_s} = \langle \mu_{ABts} | \nu \rangle + \langle \mu_{At} | \nu_{Bs} \rangle + \mathrm{interchange} (\mu, \nu)
$$

In [ ]:
S_2_ao_ = np.zeros((natm, natm, 3, 3, nao, nao))
for A in range(natm):
    sA = mol_slice(A)
    S_2_ao_[A, A, :, :, sA, :] += int1e_ipipovlp[:, :, sA, :]
    for B in range(natm):
        sB = mol_slice(B)
        S_2_ao_[A, B, :, :, sA, sB] += int1e_ipovlpip[:, :, sA, sB]
S_2_ao_ += S_2_ao_.swapaxes(-1, -2)
np.allclose(S_2_ao_, S_2_ao)

重叠积分也可以使用数值梯度的方式进行验证：

In [ ]:
np.allclose(
    NumericDiff(mol, lambda mol : HFHelper(mol).get_S_1_ao(), deriv=2).get_numdif(),
    S_2_ao
)

### Fock 矩阵二阶 Skeleton 导数 $F_{\mu \nu}^{A_t B_s}$

Yamaguchi (p408, M.7)

$$
F_{\mu \nu}^{A_t} = h_{\mu \nu}^{A_t B_s} + (\mu \nu | \kappa \lambda)^{A_t B_s} D_{\kappa \lambda} - \frac{1}{2} (\mu \kappa | \nu \lambda)^{A_t B_s} D_{\kappa \lambda}
$$

In [ ]:
F_2_ao_ = (
    + H_2_ao
    + np.einsum("ABtsuvkl, kl -> ABtsuv", eri2_ao, D)
    - 0.5 * np.einsum("ABtsukvl, kl -> ABtsuv", eri2_ao, D)
)
np.allclose(F_2_ao_, F_2_ao)

遗憾的是，目前为止我们既不能通过数值导数，也不能通过 PySCF 高层函数导出 Fock 矩阵二阶导数；事实上，因为 Fock 二阶导数几乎只有在求与二阶 CP-HF 等价的方程时才会用上，因此 PySCF 并不生成该张量．不过，等到我们成功生成 U 矩阵后，我们就可以对 Fock 一阶导数矩阵作数值完整偏导数求得二阶导数矩阵．

### ERI 二阶导数 $(\mu \nu | \kappa \lambda)^{A_t B_s}$

ERI 二阶导数从表面上，可以产生出许多项；如果我们莽撞地推导该梯度，很有可能会产生出大量的项 ($C_4^2 \times 2 + 4 = 16$ 项)．利用 ERI 的对称性，从公式推导的角度上，我们可以少些许多项；同时也有利于代码的简化．

\begin{align}
(\mu \nu | \kappa \lambda)^{A_t B_s}
&= (\partial_{A_t} \partial_{B_s} \mu \nu | \kappa \lambda)
+ (\partial_{A_t} \mu \partial_{B_s} \nu | \kappa \lambda)
+ (\partial_{A_t} \mu \nu | \partial_{B_s} \kappa \lambda)
+ (\partial_{A_t} \mu \nu | \kappa \partial_{B_s} \lambda) \\
&\quad + \mathrm{interchange} (\mu, \nu) \\
&\quad + \mathrm{interchange} (\mu \nu, \kappa \lambda) \\
&= (\mu_{ABts} \nu | \kappa \lambda)
+ (\mu_{At} \nu_{Bs} | \kappa \lambda)
+ (\mu_{At} \nu | \kappa_{Bs} \lambda)
+ (\mu_{At} \nu | \kappa \lambda_{Bs}) \\
&\quad + \mathrm{interchange} (\mu, \nu) \\
&\quad + \mathrm{interchange} (\mu \nu, \kappa \lambda)
\end{align}

In [ ]:
eri2_ao_ = np.zeros((natm, natm, 3, 3, nao, nao, nao, nao))
for A in range(natm):
    sA = mol_slice(A)
    eri2_ao_[A, A, :, :, sA, :, :, :] += int2e_ipip1[:, :, sA]
    for B in range(natm):
        sB = mol_slice(B)
        eri2_ao_[A, B, :, :, sA, sB, :, :] += int2e_ipvip1[:, :, sA, sB]
        eri2_ao_[A, B, :, :, sA, :, sB, :] += int2e_ip1ip2[:, :, sA, :, sB]
        eri2_ao_[A, B, :, :, sA, :, :, sB] += int2e_ip1ip2[:, :, sA, :, sB].swapaxes(-1, -2)
eri2_ao_ += eri2_ao_.transpose(0, 1, 2, 3, 5, 4, 6, 7)
eri2_ao_ += eri2_ao_.transpose(0, 1, 2, 3, 6, 7, 4, 5)
np.allclose(eri2_ao_, eri2_ao)

我们也能从 $\frac{\partial}{\partial B_s} (\mu \nu | \kappa \lambda)^{A_t}$ 的数值梯度得到 ERI 的二阶梯度：

In [ ]:
np.allclose(
    NumericDiff(mol, lambda mol : HFHelper(mol).get_eri1_ao(), deriv=2).get_numdif(),
    eri2_ao,
    atol=1e-7
)

<div class="alert alert-info">

**任务**

1. (可选) 我们也可以尝试对上述代码作简单的优化．我对优化过程的理解有：

    * 可以利用 $A_t$ 与 $B_s$ 的对称性
    
    * 根据 $\mu, \nu$ 和 $\mu \nu, \kappa \lambda$ 的对称性作转置加和，放在循环之外一次性求和未必是很好的选择．
   
    * 甚至转置加和本身都不是一个好的选择．你可以尝试将所有 16 项 ERI 积分展开．由于转置求和一般需要调用更大的内存空间，因此计算耗时可能更长．

</div>

## 实现参考：四脚标 $A_{pq, rs}$ 函数

### $A_{pq,rs}$

Yamaguchi (p437, X.2)

$$
A_{pq, rs} = 4 (pq | rs) - (pr | qs) - (ps | qr)
$$

但实际程序中，我们可以不使用上述的公式：我们曾经提及，生成 MO 基组的 ERI 张量是耗时的做法．因此，通常的程序都不会显式地生成 $A_{pq, rs}$ 张量，而是使用下述的公式来进行计算：

$$
A_{pq, rs} X_{rs} = C_{\mu p} C_{\nu q} \big( 2 (\mu \nu | \kappa \lambda) - (\mu \kappa | \nu \lambda) \big) \big( C_{\kappa r} X_{rs} C_{\lambda s} + C_{\kappa s} X_{rs} C_{\lambda r} \big)
$$

由此，我们就将原先 ERI 的 MO 基组化转化为了对 $X_{rs}$ 的 AO 基组化，从而降低计算复杂度．

我们现在了解 HFHelper 中提供的 `Ax0_Core` 函数的具体的调用过程．`Ax0_Core` 通过传入四个角标的分割，返回一个函数；该函数的传入参数是一个矩阵．如果现在的任务是 $A_{pi, bj} X_{bj}^{A_t}$，那么我们先向 `Ax0_Core` 传入全轨道、占据、非占、占据的分割：

In [ ]:
Ax0_Core(sa, so, sv, so)

我们发现上述函数的返回结果是一个函数内函数．随后，我们将一个末尾两个维度分别代表非占、占据的矩阵 $X_{bj}^{A_t}$ 作为参数传入上述代码的返回函数中．

In [ ]:
X = np.random.random((natm, 3, nvir, nocc))
np.allclose(
    + 4 * np.einsum("pibj, Atbj -> Atpi", eri0_mo[:, so, sv, so], X)
    - 1 * np.einsum("pbij, Atbj -> Atpi", eri0_mo[:, sv, so, so], X)
    - 1 * np.einsum("pjib, Atbj -> Atpi", eri0_mo[:, so, so, sv], X),
    Ax0_Core(sa, so, sv, so)(X)
)

刚才我们用 MO 基组的 ERI 验证了 `Ax0_Core` 函数；我们还可以使用 AO 基组 ERI 的计算过程来验证 `Ax0_Core`：

In [ ]:
dmX = Cv @ X @ Co.T
dmX += dmX.swapaxes(-1, -2)
np.allclose(
    + 2 * np.einsum("uvkl, up, vi, Atkl -> Atpi", eri0_ao, C, Co, dmX)
    - 1 * np.einsum("ukvl, up, vi, Atkl -> Atpi", eri0_ao, C, Co, dmX),
    Ax0_Core(sa, so, sv, so)(X)
)

### $A_{pq, rs}^{A_t}$

Yamaguchi (p407, M.5)

$$
A_{pq, rs}^{A_t} = 4 (pq | rs)^{A_t} - (pr | qs)^{A_t} - (ps | qr)^{A_t}
$$

这个函数很少用到，但会在生成 $B_{\mu \nu}^{A_t B_s}$ 中使用到．同刚才的 $A_{pq, rs}$ 一样，我们通常不会真的使用 MO 基组的 ERI 一阶梯度来实现 $A_{pq, rs}^{A_t} X_{rs}^{B_s}$ 的任务，而是使用 AO 基组的 ERI 一阶梯度：

In [ ]:
X = np.random.random((natm, 3, nvir, nocc))
dmX = Cv @ X @ Co.T
dmX += dmX.swapaxes(-1, -2)
np.allclose(
    + 2 * np.einsum("Atuvkl, up, vi, Bskl -> ABtspi", eri1_ao, C, Co, dmX)
    - 1 * np.einsum("Atukvl, up, vi, Bskl -> ABtspi", eri1_ao, C, Co, dmX),
    Ax1_Core(sa, so, sv, so)(X)
)

## 参考文献